In [1]:
import json, requests
import os
import numpy as np
import pandas as pd

In [2]:
API_KEY = 'JRLUCAS'

In [3]:
def get_price_history(symbols):
    candle_list = []
    df_dict = {}
    for symbol in symbols:
        API_URL = r'https://api.tdameritrade.com/v1/marketdata/{}/pricehistory'.format(symbol)
        params = {
            'apikey': API_KEY,
            'periodType': 'year',
            'period': 1,
            'frequencyType': 'daily',
            'frequency': 1,
            'needExtendedHoursData': 'false'
        }

        content = requests.get(url=(API_URL),params=params).text

        data = json.loads(content)
        candle_list.append(data)
        for i in candle_list:
            df = pd.DataFrame.from_dict(i['candles'])
            pickle_name = symbol
            pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', pickle_name + '.pickle'))
            df.to_pickle(pickle_path)

In [8]:
etf_list = ['VTI','VEA', 'VWO', 'VNQ', 'XLE', 'BND', 'SCHP', 'VTEB', 'VIG']

In [9]:
get_price_history(etf_list)

In [10]:
pickle_list = []
for etf in etf_list:
    pickle_name = etf + '.pickle'
    pickle_list.append(pickle_name)

In [11]:
df_list = []
for etf in pickle_list:
    df_name = 'df_' + etf[0:2]
    df_name = pd.read_pickle(etf)
    df_list.append(df_name)

In [14]:
for df in df_list:
    df['Normed Return'] = df['close'] / df.iloc[0]['close']
    df['Allocation'] = df['Normed Return'] * (1/9)
    pos_col = '{} Pos'.format('s')
    df[pos_col] = df['Allocation'] * 100000

In [16]:
df_list

[      close       datetime      high       low    open   volume  \
 0    142.37  1528174800000  142.5200  141.8300  142.26  2952164   
 1    143.63  1528261200000  143.6300  142.3800  142.75  3011062   
 2    143.42  1528347600000  144.0000  142.9000  143.81  1663479   
 3    143.91  1528434000000  143.9400  143.1201  143.27  1084242   
 4    144.12  1528693200000  144.4700  143.9700  144.00  1664568   
 5    144.42  1528779600000  144.6100  144.0000  144.40  1582287   
 6    143.88  1528866000000  144.6552  143.7975  144.54  1402047   
 7    144.29  1528952400000  144.5700  143.8948  144.40  1809770   
 8    144.15  1529038800000  144.3300  143.2300  143.85  3328544   
 9    143.99  1529298000000  144.0600  143.0310  143.30  1606110   
 10   143.51  1529384400000  143.6200  142.4000  142.75  1962527   
 11   143.84  1529470800000  144.1900  143.5880  143.95  1468385   
 12   142.85  1529557200000  143.8500  142.5600  143.83  1881270   
 13   142.45  1529643600000  143.0836  142.3700 

In [15]:
pos_ls = []
for k,v in df_dict.items():
    pos_ls.append(v.iloc[:,8])

NameError: name 'df_dict' is not defined

In [ ]:
portfolio_val = pd.concat(pos_ls, axis=1)

In [ ]:
portfolio_val['Total'] = portfolio_val.sum(axis=1)

In [ ]:
portfolio_val.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
portfolio_val['Total'].plot(figsize=(10,8))
plt.title('Total')

In [ ]:
portfolio_val.drop('Total',axis=1).plot(kind='line')

In [ ]:
portfolio_val['Daily Return'] = portfolio_val['Total'].pct_change(1)

In [ ]:
portfolio_val.head()

In [ ]:
portfolio_val['Daily Return'].plot(kind='kde',figsize=(4,5))

In [ ]:
cumulative_return = 100 * (portfolio_val['Total'].iloc[-1]/portfolio_val['Total'][0] - 1)

In [ ]:
cumulative_return

In [ ]:
SR = portfolio_val['Daily Return'].mean() / portfolio_val['Daily Return'].std()

In [ ]:
ASR = SR * (252**0.5)

In [ ]:
ASR